# Combined Memory

In [1]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [2]:
pip show langchain

Name: langchainNote: you may need to restart the kernel to use updated packages.

Version: 0.3.13
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\nickb\anaconda3\envs\langchain_env\Lib\site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


In [3]:
%load_ext dotenv
%dotenv

In [4]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.memory import (ConversationBufferMemory, 
                              ConversationSummaryMemory, 
                              CombinedMemory)
from langchain.globals import set_verbose

set_verbose(True)

In [5]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

C:\Users\nickb\anaconda3\envs\langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3517: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [6]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
{message_buffer_log}

Conversation summary:
{message_summary_log}

Human: {question}
AI:
'''

prompt_template = PromptTemplate.from_template(template = TEMPLATE)

In [7]:
chat_buffer_memory = ConversationBufferMemory(memory_key = 'message_buffer_log', 
                                              input_key = 'question',
                                              return_messages = False)

chat_summary_memory = ConversationSummaryMemory(llm = ChatOpenAI(), 
                                                memory_key = 'message_summary_log', 
                                                input_key = 'question',
                                                return_messages = False)

chat_combined_memory = CombinedMemory(memories = [chat_buffer_memory, 
                                                  chat_summary_memory])

C:\Users\nickb\AppData\Local\Temp\ipykernel_23596\736519893.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_buffer_memory = ConversationBufferMemory(memory_key = 'message_buffer_log',
C:\Users\nickb\AppData\Local\Temp\ipykernel_23596\736519893.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_summary_memory = ConversationSummaryMemory(llm = ChatOpenAI(),


In [11]:
print(chat_combined_memory.load_memory_variables({})['message_summary_log'])

The human asks the AI for an interesting fact, and the AI shares that octopuses have three hearts - two for the gills and one for the rest of the body.


In [9]:
chain = LLMChain(llm = chat, 
                 prompt = prompt_template, 
                 memory = chat_combined_memory)

C:\Users\nickb\AppData\Local\Temp\ipykernel_23596\1030493024.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = chat,


In [10]:
chain.invoke({'question':'''Can you give me an interesting fact I probably didn't know about?'''})
# chain.invoke({'question':'''Can you elaborate a bit more on this fact?'''})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:


Conversation summary:


Human: Can you give me an interesting fact I probably didn't know about?
AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI 

{'question': "Can you give me an interesting fact I probably didn't know about?",
 'message_buffer_log': '',
 'message_summary_log': '',
 'text': 'Sure, did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.'}